### Schools information retrieval

*Author: Liubov, Lecturers without borders*

Retrieve information about places around the point given location.
Input: Coordinates, Openstreetmap. Output: list of schools, suggested to a person. 
Packages used:

https://help.openstreetmap.org/questions/11004/download-a-map-and-extract-position-from-places-like-schools-hospitals-etc

Algorithm:
1. give coordinate (lat, lon)
2. send api call to openstreetmap 
3. get information about places around 

Alternatively if we need all the school data (~1 million of uses around the world), downloading planet file in PBF, which is only ~40 GB, and then we can filter them. 



In [8]:
import overpy


# example code to test overpy


api = overpy.Overpass()

# fetch all ways and nodes
result = api.query("""
    way(50.746,7.154,50.748,7.157) ["highway"];
    (._;>;);
    out body;
    """)

for way in result.ways:
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    print("  Nodes:")
    for node in way.nodes:
        print("    Lat: %f, Lon: %f" % (node.lat, node.lon))
        

result_school = api.query("""
    way(50.746,7.154,50.748,7.157) ["school"];
    (._;>;);
    out body;
    """)

for way in result_school.ways:
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  School: %s" % way.tags.get("school", "n/a"))
    print("  Nodes:")
        

Name: Rhedenstraße
  Highway: residential
  Nodes:
    Lat: 50.746874, Lon: 7.156307
    Lat: 50.746598, Lon: 7.156237
    Lat: 50.746477, Lon: 7.156199
    Lat: 50.746360, Lon: 7.156124
    Lat: 50.746263, Lon: 7.156023
    Lat: 50.746194, Lon: 7.155926
    Lat: 50.746139, Lon: 7.155843
Name: Von-Ketteler-Straße
  Highway: residential
  Nodes:
    Lat: 50.746905, Lon: 7.155786
    Lat: 50.747222, Lon: 7.155621
Name: Mutter-Barat-Straße
  Highway: residential
  Nodes:
    Lat: 50.747989, Lon: 7.156302
    Lat: 50.747934, Lon: 7.156229
    Lat: 50.747695, Lon: 7.155421
    Lat: 50.747605, Lon: 7.155159
    Lat: 50.747241, Lon: 7.154310
    Lat: 50.746939, Lon: 7.153634
Name: Stegerwaldstraße
  Highway: residential
  Nodes:
    Lat: 50.747853, Lon: 7.153685
    Lat: 50.747666, Lon: 7.154022
    Lat: 50.747673, Lon: 7.154122
    Lat: 50.747939, Lon: 7.154603
    Lat: 50.748182, Lon: 7.155178
    Lat: 50.748318, Lon: 7.155590
    Lat: 50.748362, Lon: 7.155646
Name: Alte Schulstraße
  Highw

### Make calls for api for schools around lat lon in given radius 
https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example#Highway_around_schools_with_inappropriate_maxspeed_.28since_0.7.54.29 

In [4]:
### Calls for API code given lat, lon 

lat = 10
lon = 100 

#// see: http://blattspinat.com/index.php/posts/tempo-50-vor-schulen

#[out:json][timeout:800];
#// Get all schools in current bounding box
#( way[amenity=school]({{bbox}});
#  node[amenity=school]({{bbox}});
#  rel[amenity=school]({{bbox}});
#)->.schools;
